<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Summarization Using Vantage and Amazon Bedrock
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;'><b>Introduction:</b></p>

<p style='font-size:16px;font-family:Arial;'>In this demo we'll deep dive on Complaints Summarization using <b>Teradata Vantage</b> and <b>Amazon Bedrock</b> model. This cutting-edge solution empowers organizations to efficiently manage and analyze customer complaints, providing actionable insights to enhance customer satisfaction and improve business operations.</p> 

<p style='font-size:16px;font-family:Arial;'><b>Key Features:</b></p> 

<ol style='font-size:16px;font-family:Arial;'>
  <li><b>AI-Powered Summarization</b>: Utilizing advanced natural language processing (NLP) and machine learning algorithms, the system automatically summarizes complaints, identifying key issues, sentiment, and root causes.</li>
  <li><b>Real-Time Analytics</b>: The platform provides real-time analytics and visualization tools, enabling users to track complaint trends, sentiment analysis, and issue resolution rates.</li>
  <li><b>Customizable Dashboards</b>: Users can create personalized dashboards to monitor specific complaint categories, product lines, or geographic regions, ensuring targeted insights and swift action.</li>
    <li><b>Integration with Amazon Bedrock</b>: Seamless integration with <b>Teradata Vantage</b> and <b>Amazon Bedrock</b> models enables users to leverage the power of cloud-based infrastructure and advanced analytics capabilities.</li>
</ol>

<p style='font-size:16px;font-family:Arial;'><b>Benefits:</b></p> 
<ol style='font-size:16px;font-family:Arial;'>
  <li><b>Enhanced Customer Experience</b>: By quickly identifying and addressing customer concerns, organizations can improve customer satisfaction and loyalty.</li>
  <li><b>Operational Efficiency</b>: Automated complaint summarization and analytics reduce manual processing time, allowing teams to focus on issue resolution and strategic decision-making.</li>
  <li><b>Data-Driven Decision-Making</b>: The platform provides actionable insights, enabling organizations to make informed decisions and drive business growth.</li>
</ol>

<p style = 'font-size:16px;font-family:Arial;'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;'>
    <li>Connect to Vantage</li>
    <li>Configuring AWS CLI</li>
    <li>Complaints Summarization</li>
    <li>Cleanup</li>
</ol>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial;'><b>Downloading and installing additional software needed</b>

In [ ]:
%%capture

!pip install --upgrade -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:2px;border:none;">
<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import numpy as np
import pandas as pd
import timeit
import boto3
from tqdm import tqdm
from teradataml import *
from datetime import datetime
import plotly.express as px

display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb

eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Complaint_Summarization.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>2. Configuring AWS CLI</b>
<p style = 'font-size:16px;font-family:Arial;'>The following cell will prompt us for the following information:</p>
<ol style = 'font-size:16px;font-family:Arial;'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>If the AWS CLI commands below fail to execute or encounter issues, you may proceed directly to Section 2.1, where the same configuration is performed using Boto3. </i></p> 

In [ ]:
# def configure_aws():
#     print("configure the AWS CLI")
#     # enter the access_key/secret_key
#     access_key = getpass.getpass("aws_access_key_id ")
#     secret_key = getpass.getpass("aws_secret_access_key ")
#     region_name = getpass.getpass("region name")

#     #set to the env
#     !aws configure set aws_access_key_id {access_key}
#     !aws configure set aws_secret_access_key {secret_key}
#     !aws configure set default.region {region_name}

In [ ]:
# does_access_key_exists = !aws configure get aws_access_key_id

# if len(does_access_key_exists) == 0:
#     configure_aws()

In [ ]:
# !aws configure list

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>2.1 Initialize the Bedrock Model</b>
<ul style = 'font-size:16px;font-family:Arial;'>
<li>The code below initializes a Boto3 client for the “bedrock-runtime” service.</li>
<li>The model can be used for natural language generation tasks.</li>
<ul>

In [ ]:
session = boto3.Session()
credentials = session.get_credentials()

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    aws_access_key_id = getpass.getpass("aws_access_key_id:"),
    aws_secret_access_key = getpass.getpass("aws_secret_access_key:"),
    aws_session_token = getpass.getpass("Enter AWS Session Token: ")
    
)

<ul style = 'font-size:16px;font-family:Arial;'>
<li>The code below tests a Boto3 client connection to the Amazon Bedrock “bedrock-runtime” service by sending a sample message to a chatbot model.</li>

In [ ]:
from botocore.exceptions import ClientError

try:
    response = client.converse(
        modelId="amazon.nova-lite-v1:0",
        messages=[{
            "role": "user",
            "content": [{"text": "Hello"}]
        }],
        inferenceConfig={"maxTokens": 10}
    )
    print("Test call successful! Response:")
    print(response)
except ClientError as e:
    print(f"Error testing client: {e}")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Complaints summarization</b>
<p style = 'font-size:16px;font-family:Arial;'>Complaints summarization with Language Model (LLM) models involves condensing lengthy complaints into concise, informative summaries. By leveraging advanced natural language processing techniques, LLMs efficiently extract key issues, sentiments, and resolutions, aiding in quicker understanding and response to customer grievances.</p>

<p style = 'font-size:16px;font-family:Arial;'>Streamlining the complaint summarization process, Language Model (LLM) models efficiently distill verbose grievances into concise, yet informative summaries. These summaries meticulously capture crucial elements including primary issues, prevalent sentiments, and possible resolutions. Harnessing advanced natural language processing capabilities, LLMs accelerate both comprehension and response to customer concerns, thereby elevating operational efficiency and bolstering overall customer satisfaction.</p>

In [ ]:
df = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints')).to_pandas(num_rows = 20)
df['Summary'] = ""
df['Reasoning with Chain of Thought'] = ""

In [ ]:
def get_summary(prompt):
    request_body = {
        "system": [
            {"text": "You are a review summarization agent"}
        ],
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt}]
            },
        ],
        "inferenceConfig": {
            "maxTokens": 300,
            "topP": 0.9,
            "topK": 20,
            "temperature": 0.7,
        }
    }
    
   # Invoke the model and extract the response body.
    response = client.invoke_model(
        modelId="amazon.nova-lite-v1:0",
        body=json.dumps(request_body)
    )
    model_response = json.loads(response["body"].read())
    return(model_response)

In [ ]:
for i in tqdm(range(len(df))):
    try:
        prompt = f'''
            The following is text from a Bank Review:

            “{df['consumer_complaint_narrative'][i]}”

            Give me reasoning as well as summary for this review.

            Instructions for Reasoning:
            - Give me Reasoning in short
            - Only one sentence reasoning
            Instructions for Summary:
            - A short one sentence Summary of everything the review states.

            Answer in the following format:
            Summary: ,
            Reasoning: ,
        '''

        response = get_summary(prompt)
        text = response['output']['message']['content'][0]['text']
        summary = re.search('Summary:(.*)', text).group(1)
        if summary == "":
            summary = re.search('Summary:\n(.*)', text).group(1)
        reasoning = re.search('Reasoning:(.*)', text).group(1)
        if reasoning == "":
            reasoning = re.search('Reasoning:\n(.*)', text).group(1)
        df['Summary'][i] = summary.strip()
        df['Reasoning with Chain of Thought'][i] = reasoning.strip()
    except:
        pass

In [ ]:
df[['complaint_id', 'consumer_complaint_narrative', 'Summary', 'Reasoning with Chain of Thought']]

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>3.1 Graph for Complaint and Summary Lengths</b></p><p style = 'font-size:16px;font-family:Arial;'>A graph illustrating the Narrative length vs summary length. On the x-axis, you'd have "Narrative length" ranging from short to long complaints or narratives. On the y-axis, you'd have "Summary length" ranging from brief to detailed summaries. As narrative length increases, summary length would generally decrease, indicating the summarization process effectively condenses longer narratives into shorter summaries. This relationship would likely follow a downward trend, showcasing the summarization efficiency of the LLM models.</p>

In [ ]:
# Truncate text for hover data
max_chars = 50  # Maximum characters to display
df['truncated_narrative'] = df['consumer_complaint_narrative'].apply(lambda x: x[:max_chars] + '...' if len(x) > max_chars else x)
df['truncated_summary'] = df['Summary'].apply(lambda x: x[:max_chars] + '...' if len(x) > max_chars else x)

# Calculate the length of consumer_complaint_narrative and Summary
df['narrative_length'] = df['consumer_complaint_narrative'].apply(len)
df['summary_length'] = df['Summary'].apply(len)

# Create a scatter plot
fig = px.scatter(df.sort_values(['narrative_length']), x='narrative_length', y='summary_length',
                 hover_data=['complaint_id', 'truncated_narrative', 'truncated_summary'],
                 labels={'narrative_length': 'Narrative Length', 'summary_length': 'Summary Length'},
                 title='Complaint and Summary Lengths')

# Update the x-axis to show values as they are (not in scientific notation)
fig.update_xaxes(type='category')

# Show the plot
fig.show(renderer="notebook")

<p style = 'font-size:16px;font-family:Arial;'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = df, table_name = 'Complaints_Summaries', if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025. All Rights Reserved
        </div>
    </div>
</footer>